In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
    



In [2]:
def data_map(wod):
    os.chdir(wod)
    labelmap={}
    labels=[]
    actions=[]
    act_vid={}
    for i in os.listdir(os.getcwd()):
        labels.append(i)
        action=os.listdir(os.path.join(wd,i))
        actions.extend(action)
        labelmap[i]=action
        for j in os.listdir(os.path.join(os.getcwd(),i)):
            act_vid[j]=os.listdir(os.path.join(os.getcwd(),i,j))
    return(labelmap,labels,actions,act_vid)

#data_map(wd)

In [3]:
# Path for exported data, numpy arrays
import os
DATA_PATH = 'C:\\Users\\sumen\\Downloads\\Final_Project\\dataset_preprocess'
wd='C:\\Users\\sumen\\Downloads\\Final_Project\\Dataset'
real_path='C:\\Users\\sumen\\Downloads\\Final_Project'
ds_path='C:\\Users\\sumen\\Downloads\\Final_Project\\dataset_final'
os.chdir(wd)

# Actions that we try to detect
labelmap,label,actions,actvid=data_map(wd)
# actions = np.array([i for i in os.listdir('C:\\Msc DS\\SignLanguageDetection\\Adjectives')])
#print(actions)

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30



In [4]:
os.chdir(ds_path)
X=np.load('sequences.npy')
y=np.load('labels.npy')

    

In [5]:
dX = np.concatenate((X,X),axis=0)
dy=np.concatenate((y,y),axis=0)

## Preprocessing

In [6]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [7]:
dy=to_categorical(dy).astype(int)

In [8]:
dy.shape

(15480, 258)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(dX, dy, test_size=0.2, random_state = 7, shuffle=True)

In [10]:
X_train.shape

(12384, 30, 1662)

In [11]:
y_test.shape

(3096, 258)

## Model Developement

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization, Dropout
from tensorflow.keras.layers import Bidirectional, LSTM
# from tensorflow.keras import Activation
import tensorflow as tf
from tf import EarlyStopping


ModuleNotFoundError: No module named 'tf'

In [13]:
from tensorflow.keras.callbacks import TensorBoard
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [14]:
model = Sequential()
model.add(Bidirectional(LSTM(256, return_sequences=True, input_shape=(30, 1662),activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01))))
model.add(BatchNormalization())
# model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Bidirectional(LSTM(128, return_sequences=True,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01))))
model.add(BatchNormalization())
# model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Bidirectional(LSTM(128, return_sequences=False,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01))))
model.add(BatchNormalization())
# model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(128, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(64, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(len(actions), activation='softmax'))

In [15]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [16]:
6
history=model.fit(X_train, y_train, epochs=300, callbacks=[tb_callback], validation_split=0.2)

Epoch 1/300
310/310 [==============================] - 175s 533ms/step - loss: 33.6095 - categorical_accuracy: 0.0041 - val_loss: 28.0081 - val_categorical_accuracy: 0.0028
Epoch 2/300
310/310 [==============================] - 169s 545ms/step - loss: 25.1358 - categorical_accuracy: 0.0063 - val_loss: 22.9691 - val_categorical_accuracy: 0.0073
Epoch 3/300
310/310 [==============================] - 166s 536ms/step - loss: 21.3600 - categorical_accuracy: 0.0121 - val_loss: 20.2493 - val_categorical_accuracy: 0.0089
Epoch 4/300
310/310 [==============================] - 165s 533ms/step - loss: 19.0602 - categorical_accuracy: 0.0157 - val_loss: 19.0819 - val_categorical_accuracy: 0.0036
Epoch 5/300
310/310 [==============================] - 164s 529ms/step - loss: 17.3959 - categorical_accuracy: 0.0165 - val_loss: 17.3482 - val_categorical_accuracy: 0.0089
Epoch 6/300
310/310 [==============================] - 167s 540ms/step - loss: 16.1125 - categorical_accuracy: 0.0228 - val_loss: 16.26

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_7 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_8 (LSTM)               (None, 64)                49408     
                                                                 
 dense_5 (Dense)             (None, 64)                4160      
                                                                 
 dense_6 (Dense)             (None, 32)                2080      
                                                                 
 dense_7 (Dense)             (None, 13)                429       
                                                                 
Total params: 597005 (2.28 MB)
Trainable params: 59700

In [ ]:
os.chdir("C:\\Users\\sumen\\Downloads\\Final_Project\\modelweights")
import pandas as pd
hist_df = pd.DataFrame(history.history) 
hist_csv_file = 'single_model_history_with_valloss_doubled.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

TypeError: 'History' object is not callable

In [ ]:
model.save('singlemodel_withvalloss_doubled.keras')

In [ ]:
train_acc=history.history['categorical_accuracy']
train_loss=history.history['loss']
val_acc=history.history['val_categorical_accuracy']
train_loss=history.history['val_loss']

In [ ]:
model.save('singlemodel_withvalloss_doubled.h5')

In [ ]:
yhat = model.predict(X_test)

1/1 [==============================] - 1s 814ms/step


In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[19,  1],
        [ 0,  0]],

       [[19,  0],
        [ 0,  1]],

       [[19,  0],
        [ 0,  1]],

       [[18,  0],
        [ 0,  2]],

       [[19,  0],
        [ 0,  1]],

       [[16,  0],
        [ 0,  4]],

       [[19,  0],
        [ 1,  0]],

       [[19,  0],
        [ 0,  1]],

       [[14,  0],
        [ 0,  6]],

       [[17,  0],
        [ 1,  2]],

       [[19,  1],
        [ 0,  0]]], dtype=int64)

In [ ]:
accuracy_score(ytrue, yhat)

0.9